# 🔥 FHIR 101 - A Practical Guide

## Notebook Setup
If you haven't already setup your development environment please do so by following the instructions on the `kf-model-fhir` GitHub repository's [README](https://github.com/kids-first/kf-model-fhir#installation)

Next, run the cell below and then refresh this page in the browser.

In [1]:
%%capture
%%bash
pip install jupyter jupyterthemes jupyter_contrib_nbextensions
jupyter contrib nbextension install
jupyter nbextension enable collapsible_headings/main
jt -t solarizedl -cursw 3 -cellw 1200 -lineh 200 -tf sourcesans

## Hello! What is this Guide?

If any of the following statements applies to you or you've had any of the questions below, then this guide is for you!
- 🤷‍♀️I've heard the term FHIR. But is it a server? A database? A data model? Rapid oxidation?
- 🙀I understand what FHIR is but have no idea how to implement it!
- 😕I tried reading through [HL7 FHIR](https://www.hl7.org/fhir/) but it is confusing and difficult to navigate
- 🤔How does FHIR modeling relate to relational databases?   

The primary purpose of this notebook is to quickly get you educated on what FHIR is and walk you through practical exercises that will teach you the FHIR basics:
1. Create and validate a FHIR data model for an entity
2. Add/remove simple and complex attributes to/from an entity in the model
3. Make entity attributes searchable in the FHIR server
4. Deploy the FHIR data model into a FHIR server
5. Load data into the FHIR server which conforms to the model
6. Search for data in the FHIR server

**If you don't care to know background and want to get right to development, head to the Tutorial section!**

## What is FHIR

Let's start with understanding exactly what FHIR is. FHIR stands for `Fast Healthcare Interoperability Resources` and was created by the organization `Health Level 7` or `HL7`.  

There is a large active international community that helps develop the specification. You can read more about it here: 

📘Learn more about [FHIR](http://hl7.org/fhir/index.html)

📘Learn more about [HL7](http://hl7.org)

### FHIR Is ...
A specification that includes all of the following:
- A loosely defined [base data model](https://www.hl7.org/fhir/resourcelist.html) describing things in healthcare (e.g. Patient, Specimen) and how they relate to     each other. The base data model also includes definitions and restrictions on server functionality
- How to extend or change the base data model to fit different healthcare use cases
- A database agnostic domain specific language (DSL) for developing files that make up the FHIR data model
- A [RESTful web API specification](https://www.hl7.org/fhir/exchange-module.html) to create, read, update, delete, and search FHIR data in a FHIR server
- A RESTful query language that describes how to construct search queries for a FHIR server
- Standards for how consumers of FHIR data should be authenticated and authorized access to health data.

### FHIR is NOT ...
- A database
- A database schema
- A server

It is important to understand that the FHIR specification is completely technology agnostic. Thus, it does not depend on programming languages or include things like relational database schemas. It is up to the implementers to decide how to implement the data model (i.e. relational database, nosql database, etc) and RESTful API. 

## Concepts

The FHIR spec is large and daunting, so we cannot cover all of the major concepts in FHIR. However, this section and the next should give you all you need to understand what you're doing in the tutorial. 

📘A good resource to learn [FHIR Basics](https://smilecdr.com/docs/tutorial_and_tour/fhir_basics.html#fhir-basics)

### FHIR Resource 

A term that describes a basic entity in the FHIR specification. A resource definition can be in JSON or XML format, but for the purposes of this guide, we will always use JSON.

📘Learn more about [FHIR Resources](https://www.hl7.org/fhir/resource.html)

**Examples of resources are:**

`Patient` - which represents a person in a healthcare system

`Specimen` - which represents a biosample drawn from a patient

`StructureDefinition` - this is different from the first two. It describes the model or definition for various
healthcare entities such as `Patient`, `Specimen`, etc.

**A `Patient` resource might look like this:**
```json
{
    "resourceType":"Patient",
    "id": "PT-001",
    "meta": {
        "profile": ["http://fhir.kids-first.io/StructureDefinition/Patient"]
    },
    "gender": "male",
    "name": [
        {
            "use": "usual",
            "family":"Smith",
            "given": "Jack",
            "period": {
                "end": "2001-01-01"
            }
        }
    ]
}
``` 

### Resource Categories

There are different categories of FHIR resources such as `Conformance Resources`, `Terminology Resources`, `Individual Resources`, `Diagnosic Resources`, etc. 

However, the most important categories of resources for the purposes of this tutorial are `Conformance Resources` and `Terminology Resources` because these are the ones we will create in order to build a FHIR data model. 

### FHIR Data Types

FHIR resource models use a set of data types for holding their values. Data types can be primitive, meaning that they hold a simple value (typically a string, a number, etc.). Other data types are composite, meaning they are a structure containing several elements. Some examples are:

**Primitive Data Types**

- String - a string of text - `female`
- Decimal - an abitrary precision float - `3.14`

**Composite Data Types**

- HumanName - A person's complete name, broken into individual parts
```json
{
   "family": "Smith",
   "given": [ "John", "Frank" ],
   "suffix": [ "Jr" ]
}
```

📘Learn about all [FHIR Data Types](https://www.hl7.org/fhir/datatypes.html)

## Model Development

What does it mean to develop a FHIR data model? Well, its not entirely different from developing any kind of data model. Developing a data model usually consists of the following steps: 

1. Define entities in the model and their attributes
2. Define relationships between entities
3. Define constraints on entity attributes (i.e. types, min/max, one of, etc)
4. Define constraints on entity relationships (i.e. Specimen must have one Patient, DiagnosticReport must have one      Patient, etc.)
5. Validate the model

More concretely, developing a data model in FHIR means:
1. Authoring JSON files containing `Conformance Resource` and `Terminology Resource` payloads
2. Validating these resources using one of the standalone HL7 validation tools or POSTing the resource payload to a      FHIR server's `/<resource type>/$validate` endpoint

### Conformance Resources

There are several different types of `Conformance Resources` but the ones that are used most in FHIR model development are: 

- `StructureDefinition` - this resource captures steps 1 through 4 above for healthcare entities like `Patient`, `Specimen`, `DiagnosticReport`, etc.

- `SearchParameter` - this resource is used to specify an attribute on an entity (Patient.gender) which can be used to search for instances of that entity on a FHIR server and what keyword/path (Patient.gender.code.value) should be used when searching.

We will take a deep dive into both of these in the tutorial.

📘Learn more about [Conformance Resources](https://www.hl7.org/fhir/conformance-module.html)

### Terminology Resources

Some entity attributes might need to be constrained to a particular value set which may come from a particular terminology system (or ontology) which defines attributes, their set of valid values, and an alphanumeric code that uniquely identifies each value.

For example, we may want to constrain the `bodySite` attribute on `Specimen` to only have values from the NCIT ontology's `Specimen Source Site` value set. We can do this by defining `Terminology Resource`s and referencing them in the `StructureDefinition` for `Specimen`.

Types of `Terminology Resource`s are:

- `CodeSystem` - describes an Ontology, Terminology, or Enumeration

- `ValueSet` - specifies a set of code and values drawn from one or more code systems

You will see a practical example of this later in the tutorial.

📘Learn about [Terminology Resources](https://www.hl7.org/fhir/terminologies.html)

## Server Development

### Pick an Existing Solution
You might have noticed that there are not many open source or free FHIR servers currently in development. This is likely due to a couple of factors:

- The FHIR specification is rapidly evolving and the tooling around it is still in early stages of development. 
- Server development is hard 

In order to develop a compliant FHIR server you must: 
- Implement the FHIR model validation logic. This is not easy since a FHIR model is authored in a custom DSL.           Server must know how to parse conformance resources and execute rules defined in them.
- Implement a parser and query constructor for the custom FHIR RESTful query language

### FHIR Server Solutions
For these reasons it is best to select one of the existing FHIR server solutions rather than develop your own. The most promising ones are:

<table style="bgcolor: none;">
    <tbody>
        <tr>
            <td style="border-right: 1px solid #cccccc; padding: 7px;">
                <a href="https://health-samurai.webflow.io/aidbox">
                    <img src="static/images/aidbox-logo.png" alt="Health Samurai Aidbox"/>
                </a>
            </td>
            <td style="padding: 7px;">
                <a href="https://smilecdr.com">
                    <img src="static/images/smilecdr-logo.png" alt="Smile CDR"/>
                </a>
            </td>
        </tr>
    </tbody>
</table>





## Tutorial - TODO

- Create a Patient Model
- Remove an Attribute
- Validate the Patient Model
- Create a Patient Resource
- Load the Patient Resource
- Add an Attribute
- Make an Attribute Searchable
- Add a Coded Attribute
- Documenting the Model
- Publish Model Documentation